<a href="https://colab.research.google.com/github/AbhinavJain3/IntelliQuery/blob/main/IntelliQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install PyPDF2 nltk pinecone-client sentence-transformers google-cloud-aiplatform fastapi uvicorn google-generativeai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.7 MB/s eta 0:00:00


In [ ]:
# First, let's install the required libraries
# !pip install PyPDF2 nltk

import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from PyPDF2 import PdfReader
from google.colab import files

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text() + '\n'
    return text

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Join the tokens back into a string
    cleaned_text = ' '.join(tokens)

    return cleaned_text

# Upload PDF file
uploaded = files.upload()

# Process the uploaded file
for filename in uploaded.keys():
    # Extract text from the PDF
    extracted_text = extract_text_from_pdf(filename)

    # Preprocess the extracted text
    preprocessed_text = preprocess_text(extracted_text)

    # Print some statistics
    print(f"File: {filename}")
    print(f"Original text length: {len(extracted_text)} characters")
    print(f"Preprocessed text length: {len(preprocessed_text)} characters")

    # Print a sample of the preprocessed text (first 500 characters)
    print("\nSample of preprocessed text:")
    print(preprocessed_text[:500] + "...")

    # Save the preprocessed text to a file
    output_filename = f"{os.path.splitext(filename)[0]}_preprocessed.txt"
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(preprocessed_text)
    print(f"\nPreprocessed text saved to: {output_filename}")

    # Provide download link for the preprocessed file
    files.download(output_filename)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Saving iesc111.pdf to iesc111.pdf
File: iesc111.pdf
Original text length: 32950 characters
Preprocessed text length: 21887 characters

Sample of preprocessed text:
everyday hear sounds various sources like humans bir ds bells machines vehicles televisions radios etc sound form ener gy pr oduces sensation hearing ears ther e also forms energy like mechanical energy light energy etc w e talked mechanical energy pr evious chapters ou taught conservation energy states neither create destr oy ener gy w e change fr om one another clap sound produced produce sound without utilising energy form energy use produce sound chapter going learn sound produced transmitte...

Preprocessed text saved to: iesc111_preprocessed.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# !pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
import pinecone
from sentence_transformers import SentenceTransformer
import torch
from google.colab import files

# Initialize Pinecone client
api_key = "YOUR-PINECONE-API-KEY"
pc = pinecone.Pinecone(api_key=api_key, environment="us-east-1")

# Define index specifications
from pinecone import ServerlessSpec  # Import ServerlessSpec or PodSpec as needed

index_name = "ncert-index"
if index_name not in [i['name'] for i in pc.list_indexes()]:  # Adjusted to match list_indexes response format
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # Adjust as needed
        )
    )

# Connect to the index
index = pc.Index(index_name)

# Upload the preprocessed text file
uploaded = files.upload()

# Get the filename of the uploaded file
filename = next(iter(uploaded))

# Load the preprocessed text
with open(filename, 'r', encoding='utf-8') as f:
    preprocessed_text = f.read()

# Split the text into chunks (adjust chunk_size as needed)
chunk_size = 1000
chunks = [preprocessed_text[i:i + chunk_size] for i in range(0, len(preprocessed_text), chunk_size)]

# Initialize the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings and store in Pinecone
batch_size = 32
for i in range(0, len(chunks), batch_size):
    # Get a batch of chunks
    batch = chunks[i:i + batch_size]

    # Create embeddings
    embeddings = model.encode(batch)

    # Prepare the batch for upsert
    ids = [f"chunk_{j}" for j in range(i, i + len(batch))]
    vectors = [(ids[k], embeddings[k].tolist(), {"text": batch[k]}) for k in range(len(batch))]

    # Upsert to Pinecone
    index.upsert(vectors=vectors)

print(f"Uploaded {len(chunks)} chunks to Pinecone index '{index_name}'")



Saving iesc111_preprocessed.txt to iesc111_preprocessed (1).txt


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Uploaded 22 chunks to Pinecone index 'ncert-index'


In [ ]:
import pinecone
from sentence_transformers import SentenceTransformer
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Initialize Pinecone client
api_key = "YOUR-PINECONE-API-KEY"
pc = pinecone.Pinecone(api_key=api_key, environment="us-east-1")

# Connect to the Pinecone index
index_name = "ncert-index"
index = pc.Index(index_name)

# Initialize the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define and preprocess your query text
query_text = "what are infrasonics?"

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Process the query text
query_text = preprocess_text(query_text)

# Create the query embedding
query_embedding = model.encode([query_text])[0]

# Perform the query
query_response = index.query(
    vector=query_embedding.tolist(),  # Ensure embedding is a list
    top_k=5,  # Number of similar items to retrieve
    include_values=False,  # Set to True if you need the vector values
    include_metadata=True  # Set to False if you do not need metadata
)

# Print the results
print("Query Results:")
for match in query_response['matches']:
    print(f"ID: {match['id']}, Score: {match['score']}")
    if 'metadata' in match:
        print(f"Metadata: {match['metadata']}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Query Results:
ID: chunk_14, Score: 0.320274115
Metadata: {'text': 'ns wings bee rhinoceroses communicate using infrasound frequency low hz whales elephants produce sound infrasound range observed animals get disturbed earthquakes earthquakes produce lowfrequency infrasound main shock waves begin possibly alert animals frequencies higher khz called ultrasonic sound ultrasound ultrasound produced animals dolphins bats porpoises moths certain families sensitive hearing equipment moths hear high frequencyq fig stethoscope generally ceilings concert halls conference halls cinema halls curved sound reflection reaches corners hall shown fig sometimes curved soundboard may placed behind stage sound reflecting sound board spreads evenly across width hall fig instruments tube followed conical opening reflects sound successively guide sound waves source forward direction towards audience rationalised science squeaks bat know bat flying nearby able escape capture rats also play games producing ul

In [ ]:
import pinecone
from sentence_transformers import SentenceTransformer
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Initialize Pinecone client
api_key = "YOUR-PINECONE-API-KEY"
pc = pinecone.Pinecone(api_key=api_key, environment="us-east-1")

# Connect to the Pinecone index
index_name = "ncert-index"
index = pc.Index(index_name)

# Initialize the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define and preprocess your query text
query_text = "What is the speed of sound in different mediums?"

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Process the query text
query_text = preprocess_text(query_text)

# Create the query embedding
query_embedding = model.encode([query_text])[0]

# Perform the query
query_response = index.query(
    vector=query_embedding.tolist(),  # Ensure embedding is a list
    top_k=5,  # Number of similar items to retrieve
    include_values=False,  # Set to True if you need the vector values
    include_metadata=True  # Set to False if you do not need metadata
)

# Print the results
print("Query Results:")
for match in query_response['matches']:
    print(f"ID: {match['id']}, Score: {match['score']}")
    if 'metadata' in match:
        print(f"Metadata: {match['metadata']}")


Query Results:
ID: chunk_11, Score: 0.590944529
Metadata: {'text': 'ound propagates medium finite speed sound thunder heard little later flash light seen make sound travels speed much less speed light speed sound depends properties medium thr ough travels ou learn dependence higher classes speed sound medium depends temperature medium speed sound decreases go solid gaseous state medium increase temperature speed sound increases example speed sound air c c speeds sound particular temperature various media ar e listed able ou need memorise values table speed sound different media c state substance speed ms solids aluminium nickel steel iron brass glass flint liquids water sea water distilled ethanol methanol gases hydrogen helium air oxygen sulphur dioxide fig reflection sound rationalised science echo shout clap near suitable reflecting object tall building mountainwe hear sound little later sound hear called echo sensation sound persists brain hear distinct echo time interval original 

In [ ]:
import pinecone
from sentence_transformers import SentenceTransformer

# Initialize Pinecone client
api_key = "YOUR-PINECONE-API-KEY"
pc = pinecone.Pinecone(api_key=api_key, environment="us-east-1")

# Connect to the existing index
index_name = "ncert-index"
index = pc.Index(index_name)

# Initialize the embedding model (if you need it for querying)
model = SentenceTransformer('all-MiniLM-L6-v2')

print(f"Successfully connected to Pinecone index '{index_name}'")

# Optional: Check index stats
print(index.describe_index_stats())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Successfully connected to Pinecone index 'ncert-index'
{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 22}},
 'total_vector_count': 22}


In [ ]:
import os
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
import google.generativeai as genai

# Initialize Pinecone
api_key = "YOUR-PINECONE-API-KEY"
pc = Pinecone(api_key=api_key)
index_name = "ncert-index"
index = pc.Index(index_name)

# Initialize SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize Google Gemini API
genai.configure(api_key="YOUR-GOOGLE-GEMINI-API-KEY")
gemini_model = genai.GenerativeModel('gemini-pro')

def query_index(query_text):
    query_embedding = model.encode([query_text])[0].tolist()
    query_response = index.query(
        vector=query_embedding,
        top_k=5,
        include_metadata=True
    )
    return query_response

def prepare_context(query_response):
    context = ""
    for match in query_response['matches']:
        context += f"{match['metadata']['text']}\n\n"
    return context.strip()

def generate_response(query, context):
    prompt = f"""Based on the following context, answer the question: "{query}"

Context:
{context}

Answer:"""
    response = gemini_model.generate_content(prompt)
    return response.text

def process_query(query_text):
    query_response = query_index(query_text)
    context = prepare_context(query_response)
    response = generate_response(query_text, context)
    return response

# Example usage
if __name__ == "__main__":
    query = "Characteric of sound wave?"
    answer = process_query(query)
    print(f"Query: {query}")
    print(f"Answer: {answer}")

Query: Characteric of sound wave?
Answer: Wavelength, frequency, and amplitude
